In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import argparse
import cv2
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

paths=[]
labels=[]

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths+=[os.path.join(dirname, filename)]
        labels+=[dirname.split('/')[-1]]

print(paths[0:3])

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['/kaggle/input/eye-diseases-classification/dataset/glaucoma/1255_right.jpg', '/kaggle/input/eye-diseases-classification/dataset/glaucoma/1482_left.jpg', '/kaggle/input/eye-diseases-classification/dataset/glaucoma/_398_8823411.jpg']


In [4]:
data_df=pd.DataFrame(columns=['path','label'])
data_df['path']=paths
data_df['label']=labels
data_df = data_df.sample(frac=1).reset_index(drop=True)

# display(data_df)
# print(len(data_df))
display(data_df['label'].value_counts())

diabetic_retinopathy    1098
normal                  1074
cataract                1038
glaucoma                1007
Name: label, dtype: int64

In [5]:
train_data_df, test_data_df = train_test_split(data_df, test_size=0.2)

In [6]:
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure

In [8]:
def read_image_for_hog(path):
    img = imread(path)
#     plt.axis("off")
#     plt.imshow(img)
#     print(img.shape)
    
    return img


def resize_image_for_hog(input_img):
    resized_img = resize(input_img, (128*4, 64*4))
#     plt.axis("off")
#     plt.imshow(resized_img)
#     print(resized_img.shape)
    
    return resized_img


def extract_features_using_hog(resized_image):
    fd, hog_image = hog(resized_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, channel_axis=-1)
#     print((hog_image))
#     print("fd:", fd)
#     plt.axis("off")
#     plt.imshow(hog_image, cmap="gray")
#     plt.show()
    
    return fd, hog_image

In [9]:
def extract_features_HoG(img_paths_df):
    data = []
    labels = []
    
    counter = 1
    for i in img_paths_df.index:
        img_path = img_paths_df["path"][i]
        label = img_paths_df["label"][i]
        print(counter, ". ",img_path, " | ", label)
        
        img = read_image_for_hog(img_path)
        resized_img = resize_image_for_hog(img)
        fd, hog_image = extract_features_using_hog(resized_img)
        
        labels.append(label)
        data.append(hog_image)
        counter += 1
    
    return labels, data

In [ ]:
img_labels_hog, img_data_hog = extract_features_HoG(train_data_df)

In [ ]:
img_labels_test_hog, img_data_test_hog = extract_features_HoG(test_data_df)

In [ ]:
from sklearn import svm

model = LinearSVC(dual=False, C=100.0, random_state=42)
model.fit(img_data_hog, img_labels_hog)
pred = model.predict(img_data_test_hog)

from sklearn import metrics
print("Feature Extraction using: Histogram of Oriented Gradients (HOG)\nAccuracy:", round(metrics.accuracy_score(img_labels_test_hog, y_pred=pred) * 100,3),"%")

# linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(img_data_hog, img_labels_hog)
# rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(img_data_hog, img_labels_hog)
# poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(img_data_hog, img_labels_hog)
# sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(img_data_hog, img_labels_hog)